In [1]:
!pip install fastapi uvicorn pyngrok joblib


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.2/92.2 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.4/341.4 kB 22.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 45.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 32.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 7.0 MB/s eta 0:00:00


In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
import joblib
data = pd.read_csv('predata.csv')
# Assuming the data has columns ['feature1', 'feature2', ..., 'target']
features = ['TransactionId', 'BatchId', 'AccountId', 'SubscriptionId', 'CustomerId', 'CountryCode',
            'ProviderId', 'ProductId', 'ChannelId', 'Amount', 'Value',
            'PricingStrategy', 'TransactionHour', 'TransactionDay', 'TransactionMonth', 'TransactionYear',
            'ProductCategory_data_bundles', 'ProductCategory_financial_services',
            'ProductCategory_movies', 'ProductCategory_other', 'ProductCategory_ticket',
            'ProductCategory_transport', 'ProductCategory_tv', 'ProductCategory_utility_bill']

target = 'FraudResult'  # Replace with your actual target column name

# Split the data into features and target
X = data[features]
y = data[target]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the XGBClassifier model
model = XGBClassifier()
model.fit(X_train, y_train)

# Save the model to a file
joblib.dump(model, 'XGBClassifier_model.pkl')

['XGBClassifier_model.pkl']

In [7]:
%%writefile app.py
from fastapi import FastAPI
import joblib
import pandas as pd
from pydantic import BaseModel

app = FastAPI()

# Load the model
model = joblib.load('xgb_model.pkl')

# Define the request body
class Transaction(BaseModel):
    TransactionId: int
    BatchId: int
    AccountId: int
    SubscriptionId: int
    CustomerId: int
    CountryCode: int
    ProviderId: int
    ProductId: int
    ChannelId: int
    Amount: float
    Value: float
    PricingStrategy: int
    TransactionHour: int
    TransactionDay: int
    TransactionMonth: int
    TransactionYear: int
    ProductCategory_data_bundles: int
    ProductCategory_financial_services: int
    ProductCategory_movies: int
    ProductCategory_other: int
    ProductCategory_ticket: int
    ProductCategory_transport: int
    ProductCategory_tv: int
    ProductCategory_utility_bill: int

@app.post("/predict")
def predict(transaction: Transaction):
    # Convert the request data to a DataFrame
    data = pd.DataFrame([transaction.dict().values()], columns=transaction.dict().keys())

    # Make a prediction
    prediction = model.predict(data)
    return {"prediction": int(prediction[0])}

if __name__ == "__main__":
    import uvicorn
    uvicorn.run(app, host="0.0.0.0", port=8000)


Writing app.py


In [10]:
!ngrok kill


ngrok - tunnel local ports to public URLs and inspect traffic

USAGE:
  ngrok [command] [flags]

AUTHOR:
  ngrok - <support@ngrok.com>

COMMANDS: 
  config          update or migrate ngrok's configuration file
  http            start an HTTP tunnel
  tcp             start a TCP tunnel
  tunnel          start a tunnel for use with a tunnel-group backend

EXAMPLES: 
  ngrok http 80                                                 # secure public URL for port 80 web server
  ngrok http --domain baz.ngrok.dev 8080                        # port 8080 available at baz.ngrok.dev
  ngrok tcp 22                                                  # tunnel arbitrary TCP traffic to port 22
  ngrok http 80 --oauth=google --oauth-allow-email=foo@foo.com  # secure your app with oauth

Paid Features: 
  ngrok http 80 --domain mydomain.com                           # run ngrok with your own custom domain
  ngrok http 80 --allow-cidr 2600:8c00::a03c:91ee:fe69:9695/32  # run ngrok with IP policy restrictions

In [11]:
from pyngrok import ngrok
import subprocess
import time

# Install necessary packages
!pip install fastapi uvicorn pyngrok

# Create a FastAPI app
with open("app.py", "w") as f:
    f.write('''
from fastapi import FastAPI
import joblib
import pandas as pd
from pydantic import BaseModel

app = FastAPI()

# Load the model
model = joblib.load('XGBClassifier_model.pkl')

# Define the request body
class Transaction(BaseModel):
    TransactionId: int
    BatchId: int
    AccountId: int
    SubscriptionId: int
    CustomerId: int
    CountryCode: int
    ProviderId: int
    ProductId: int
    ChannelId: int
    Amount: float
    Value: float
    PricingStrategy: int
    TransactionHour: int
    TransactionDay: int
    TransactionMonth: int
    TransactionYear: int
    ProductCategory_data_bundles: int
    ProductCategory_financial_services: int
    ProductCategory_movies: int
    ProductCategory_other: int
    ProductCategory_ticket: int
    ProductCategory_transport: int
    ProductCategory_tv: int
    ProductCategory_utility_bill: int

@app.post("/predict")
def predict(transaction: Transaction):
    # Convert the request data to a DataFrame
    data = pd.DataFrame([transaction.dict().values()], columns=transaction.dict().keys())

    # Make a prediction
    prediction = model.predict(data)
    return {"prediction": int(prediction[0])}

if __name__ == "__main__":
    import uvicorn
    uvicorn.run(app, host="0.0.0.0", port=8000)
    ''')

# Function to stop all existing ngrok tunnels
def stop_all_tunnels():
    tunnels = ngrok.get_tunnels()
    for tunnel in tunnels:
        ngrok.disconnect(tunnel.public_url)

# Set up ngrok config
ngrok.set_auth_token("2jPuTQbl3DRbRcgOTXsQ8GFjCLX_3tBPHUHpgbocMqoSmQ8d")

# Stop all existing tunnels
stop_all_tunnels()

# Start the FastAPI server
process = subprocess.Popen(["uvicorn", "app:app", "--host", "0.0.0.0", "--port", "8000", "--reload"])

# Give the server time to start
time.sleep(10)

# Set up ngrok tunnel
public_url = ngrok.connect(8000).public_url
print("ngrok tunnel available at:", public_url)

# Now you can test your API
import requests

url = f"{public_url}/predict"
payload = {
    "TransactionId": 1,
    "BatchId": 1,
    "AccountId": 1,
    "SubscriptionId": 1,
    "CustomerId": 1,
    "CountryCode": 1,
    "ProviderId": 1,
    "ProductId": 1,
    "ChannelId": 1,
    "Amount": 1,
    "Value": 1,
    "PricingStrategy": 1,
    "TransactionHour": 1,
    "TransactionDay": 1,
    "TransactionMonth": 1,
    "TransactionYear": 1,
    "ProductCategory_data_bundles": 1,
    "ProductCategory_financial_services": 1,
    "ProductCategory_movies": 1,
    "ProductCategory_other": 1,
    "ProductCategory_ticket": 1,
    "ProductCategory_transport": 1,
    "ProductCategory_tv": 1,
    "ProductCategory_utility_bill": 1
}

# Make the request
response = requests.post(url, json=payload)

try:
    print(response.json())
except requests.exceptions.JSONDecodeError:
    print("Error decoding JSON response. The response was:", response.text)


ngrok tunnel available at: https://ac05-34-73-49-43.ngrok-free.app
{'prediction': 0}
